In [ ]:
class WebScraperAgent:
    def __init__(self):
        self.playwright = None
        self.browser = None
        self.page = None

    async def init_browser(self):
        try:
            self.playwright = await async_playwright().start()
            self.browser = await self.playwright.chromium.launch(
                headless=True,
                args=[
                    "--disable-dev-shm-usage",
                    "--no-sandbox",
                    "--disable-setuid-sandbox",
                    "--disable-gpu",  # Simplified args
                ]
            )
            self.page = await self.browser.new_page()
            return True
        except Exception as e:
            print(f"❌ Browser initialization failed: {str(e)}")
            await self.safe_close()
            return False

    async def safe_close(self):
        """Safe cleanup that handles partial initialization"""
        try:
            if hasattr(self, 'page') and self.page and not self.page.is_closed():
                await self.page.close()
            if hasattr(self, 'browser') and self.browser:
                await self.browser.close()
            if hasattr(self, 'playwright') and self.playwright:
                await self.playwright.stop()
        except Exception as e:
            print(f"⚠️ Cleanup error: {str(e)}")
        finally:
            self.page = None
            self.browser = None
            self.playwright = None

    async def scrape_content(self, url):
        if not await self.init_browser():
            raise RuntimeError("Browser not initialized")
        
        try:
            await self.page.goto(url, wait_until="domcontentloaded")
            await self.page.wait_for_load_state("networkidle", timeout=5000)
            return await self.page.content()
        except Exception as e:
            print(f"❌ Scraping failed: {str(e)}")
            await self.safe_close()
            raise

async def webscraper(target_url, instructions):
    scraper = WebScraperAgent()
    try:
        print("Initializing browser...")
        if not await scraper.init_browser():
            return None, None

        print("Extracting HTML content...")
        html_content = await scraper.scrape_content(target_url)

        print("Taking screenshot...")
        screenshot = await scraper.page.screenshot(type="png")

        print("Processing with LLM...")
        result = await process_with_llm(html_content, instructions, False)
        print("✅ Done!")
        return result, screenshot
        
    except Exception as e:
        print(f"❌ Fatal error: {str(e)}")
        return None, None
    finally:
        await scraper.safe_close()